In [31]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import Layout, HBox, interactive, IntSlider, FloatSlider, Checkbox, Dropdown, widgets, VBox, Button, widgets, HTML
from sklearn import datasets
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import time
import bqplot
import threading

plt.rcParams['figure.figsize'] = (6.0,6.0)
%matplotlib inline

In [2]:
class Dataset:
    X = None
    y = None
    X_train = None
    X_test = None
    y_train = None
    y_test = None
    
    def __init__(self,*args):
        self.updateData(*args)
    
    def _split_data(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y)
        
    def getAllData(self):
        return [self.X,self.y]
    
    def getTrainData(self):
        return [self.X_train, self.y_train]
    
    def getTestData(self):
        return [self.X_test, self.y_test]
    
    def updateData(self,*args):
        self.X = args[0][0]
        self.y = args[0][1]
        if isinstance(self.X,np.ndarray) and isinstance(self.y,np.ndarray) \
            and self.X.size != 0 and self.y.size != 0:
            self.y = self.y.reshape(-1,1)
            self._split_data()
        return self

In [3]:
#Util Functions
def relu(z):
    z[z<0] = 0
    return z
  

def drelu(a):
    a[a>0] = 1
    return a


def sigmoid(z):
    return np.divide(1,(1+np.power(np.e,-z)))


def dsigmoid(a):
    return a * (1-a)


def calculateAccuracy(y, a):
    return np.squeeze(np.dot(y.T, a) + np.dot((1 - y).T, (1 - a))) / y.shape[0] * 100


def calculateCrossentropyCost(y, a, noOfTrainingSamples, weights=0):
    cost = (-1 / noOfTrainingSamples) * (np.dot(y.T, np.log(a)) + np.dot((1 - y).T, np.log(1 - a)))
    return np.squeeze(cost)

In [4]:
def buildNetwork(self):

    """
    Function to build the network architecture using initialized values
    """

    inputDims = self.inputSize[1]
    outputClasses = 1 #self.outputSize[1]
    
    #If there are any hidden layers at all
    if self.hiddenLayerSize:
        #Initialize weights between Input Layer and first hidden layer
        self.weight_dict['12'] = np.random.randn(inputDims, self.hiddenLayerSize[0]) * 0.01
        self.bias_dict['12'] = np.zeros((1, self.hiddenLayerSize[0]))

        #Initialize weights between Last Hidden Layer and output layer
        self.weight_dict[str(self.noOfLayers - 1) + str(self.noOfLayers)] = np.random.randn(self.hiddenLayerSize[-1], outputClasses) * 0.01
        self.bias_dict[str(self.noOfLayers - 1) + str(self.noOfLayers)] = np.zeros((1, outputClasses))

        for x in range(2, self.noOfLayers - 1):
            self.weight_dict[str(x) + str(x + 1)] = np.random.randn(self.hiddenLayerSize[x - 2], self.hiddenLayerSize[x - 1]) * 0.01
            self.bias_dict[str(x) + str(x + 1)] = np.zeros((1, self.hiddenLayerSize[x - 1]))
    #Just logistic regression model with input and output nodes
    else:
        self.weight_dict['12'] = np.random.randn(inputDims, outputClasses) * 0.01
        self.bias_dict['12'] = np.zeros((1, outputClasses)) 
    self.readyToTrain = True

In [5]:
def forwardPass(self):
    """
    Function to start the forward pass by passing the Input as a batch. Calls calculate layer errors
    immediately after pass is over to store layer errors for each node.

    """
    self.op = self.X
    self.activation_dict['1'] = self.X

    for layer in range(1, self.noOfLayers):
        #print("Enter op size: {}".format(self.op.shape))
        z = np.dot(self.op, self.weight_dict[str(layer) + str(layer + 1)]) + self.bias_dict[str(layer) + str(layer + 1)]
        if layer != self.noOfLayers - 1:
            self.op = relu(z)
            self.dactivation_dict[str(layer + 1)] = drelu(self.op)
            self.activation_dict[str(layer + 1)] = self.op
        else:
            self.op = sigmoid(z)
            self.dactivation_dict[str(layer + 1)] = dsigmoid(self.op)
            self.activation_dict[str(layer + 1)] = self.op

    self.calculate_layer_errors()
        #print("Exit op size: {}".format(self.op.shape))

In [6]:
def calculate_layer_errors(self):
    """
    Function to calculate layer errors for each node in each layers. Usually called after forward pass
    automatically
    """

    for layer in reversed(range(1, self.noOfLayers)):
        if layer == self.noOfLayers - 1:
            self.error_dict[str(layer + 1)] = self.op - self.y.reshape(-1, 1)
            continue

        self.error_dict[str(layer + 1)] = np.dot(self.error_dict[str(layer + 2)],
                                        self.weight_dict[str(layer + 1) + str(layer + 2)].T) \
                                        * self.dactivation_dict[str(layer + 1)]

In [7]:
def calculate_gradients(self):
    """
    Function to calculate gradients of each layer from the layer errors calculated after the forward pass.
    This function is called by update weights to calculate gradients and update the weights by substracting
    them from the old weights.
    """
    for layer in range(1, self.noOfLayers):
        key = str(layer) + str(layer+1)
        self.dW[key] = 1 / self.noOfTrainingSamples * np.dot(self.activation_dict[str(layer)].T,self.error_dict[str(layer+1)])
        self.dB[key] = 1 / self.noOfTrainingSamples * np.sum(self.error_dict[str(layer + 1)], axis=0, keepdims=True)


In [8]:
def update_weights(self):
        """
        Function to calculate gradients and substract them from old weights
        """
        self.calculate_gradients()
        updated_weights = {}
        updated_bias = {}
        for key in self.weight_dict.keys():
            updated_weights[key] = self.weight_dict[key] - self.learning_rate * self.dW[key]
            updated_bias[key] = self.bias_dict[key] - self.learning_rate * self.dB[key]

        self.weight_dict = updated_weights
        self.bias_dict = updated_bias

In [9]:
def predict(self,X_test):
    """
    Predicts the output of the new set of input values that need to be validated.
    """
    op = X_test
    for layer in range(1, self.noOfLayers):
        z = np.dot(op, self.weight_dict[str(layer) + str(layer + 1)]) + self.bias_dict[str(layer) + str(layer + 1)]
        if layer == self.noOfLayers-1:
            op = sigmoid(z)
            return op

        op = relu(z)

In [1]:
def showNetworkArch(self, left=.1, right=.9, bottom=.1, top=.9):
    '''
    Draw a neural network cartoon using matplotilb.
    
    :usage:
        >>> fig = plt.figure(figsize=(12, 12))
        >>> draw_neural_net(fig.gca(), .1, .9, .1, .9, [4, 7, 2])
    
    :parameters:
        - ax : matplotlib.axes.AxesSubplot
            The axes on which to plot the cartoon (get e.g. by plt.gca())
        - left : float
            The center of the leftmost node(s) will be placed here
        - right : float
            The center of the rightmost node(s) will be placed here
        - bottom : float
            The center of the bottommost node(s) will be placed here
        - top : float
            The center of the topmost node(s) will be placed here
        - layer_sizes : list of int
            List of layer sizes, including input and output dimensionality
    '''
    fig = plt.figure(figsize=(6,6))
    ax = fig.gca()
    ax.axis('off')
    
    layer_sizes = (self.inputSize[1],*self.hiddenLayerSize,self.outputSize[1]) if self.hiddenLayerSize else (self.inputSize[1],self.outputSize[1])
    n_layers = len(layer_sizes)
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in range(layer_size):
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                                color='w', ec='k', zorder=4)
            ax.add_artist(circle)
    # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in range(layer_size_a):
            for o in range(layer_size_b):
                line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                  [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], c='k')
                ax.add_artist(line)
    plt.show();

In [11]:
def startTraining(self,epochs = 1000, verbose=False):
    
    if self.X.size == 0 or self.y.size == 0:
        print("No input/output data provided to train the model against. Please provide atleast one row :)")
        return
    if not self.readyToTrain:
        print("Network architecture is not yet build. Please build one first :)")
        return

    self.cost_array = []
    self.accuracy_array = []
    epoch = 1
    while self.readyToTrain and epoch <= epochs:
        self.forwardPass()
        self.cost_array.append(calculateCrossentropyCost(self.y, self.op, self.noOfTrainingSamples, self.weight_dict))
        self.accuracy_array.append(calculateAccuracy(self.y,self.op))
        self.update_weights()
        
        if not epoch % 10 and verbose: 
            avg_cost = sum(self.cost_array[epoch-10:epoch])/10
            avg_acc = sum(self.accuracy_array[epoch-10:epoch])/10
            print(f"EPOCH: {epoch} :: AVG. COST: {avg_cost} :: AVG. ACCURACY: {avg_acc}%")
        time.sleep(0.01)
        epoch+=1

In [12]:
def injectTrainData(self,X,y):
    if X.size == 0 or y.size == 0:
        print("No input/output data provided to train the model against. Please provide atleast one row :)")
        return
    if y.shape[0] != X.shape[0]:
        print("Number of samples for X and y don't match. Please provide valid data")
        return
    if X.shape != self.inputSize:
        print("Network Architecture and training data shapes do not match. Please provide valid data")
        return
    self.X = X
    self.y = y

In [13]:
def reloadNetwork(self,data):
    if isinstance(data,Dataset):
        X,y = data.getAllData()
        self.inputSize, self.outputSize= [arr.shape for arr in data.getTrainData()]

In [14]:
def plot_decision_boundary(self):
    fig, ax = plt.subplots(1,1)
    x_min,x_max = X[:,0].min()-1,X[:,0].max() + 1
    y_min, y_max  = X[:,1].min() -1 , X[:,1].max() +1
    distance = 0.01

    xx,yy = np.meshgrid(np.arange(x_min,x_max,distance),np.arange(y_min,y_max,distance))
    Z = self.predict(np.c_[xx.ravel(),yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contourf(xx,yy,Z,cmap=plt.cm.Spectral)
    ax.scatter(X[:,0], X[:,1],y,cmap='viridis')
    ax.show();

In [2]:
class NeuralNet():

    weight_dict = {}
    bias_dict = {}
    activation_dict = {}
    dactivation_dict = {}
    error_dict = {}
    dW = {}
    dB = {}
    op = None
    readyToTrain = False
    noOfTrainingSamples = 0
    X=None
    y=None
    cost_array = []
    accuracy_array = []


    def __init__(self,inputSize=(500,2),outputSize=(500,),hiddenLayerSize=(4,),learning_rate=0.06):
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenLayerSize = hiddenLayerSize
        self.noOfLayers = len(self.hiddenLayerSize) + 2
        self.learning_rate = learning_rate
        self.noOfTrainingSamples = inputSize[0]
        self.epoch = 1
    
    def buildNetwork(self):
        buildNetwork(self)

    def forwardPass(self):
        forwardPass(self)

    def calculate_layer_errors(self):
        calculate_layer_errors(self)

    def calculate_gradients(self):
        calculate_gradients(self)

    def update_weights(self):
        update_weights(self)

    def predict(self,X_test):
        return predict(self,X_test)

    def showNetworkArch(self):
        showNetworkArch(self)
        
    def startTraining(self,epochs=1000,verbose=False):
        return startTraining(self, epochs, verbose)
        
    def reloadNetwork(self,data):
        reloadNetwork(self,data)
        
    def setHiddenLayerSize(self,hiddenLayerSize):
        self.hiddenLayerSize = hiddenLayerSize

    def injectTrainData(self, X=None, y=None):
        injectTrainData(self, X, y)

In [51]:
data = Dataset(datasets.make_moons(800, noise=0.15,shuffle=True))
net = NeuralNet()
epoch_flag = False

In [52]:
#Change dataset over here
#make moons dataset
def create_dataset_widget():
    def update_dataset_plot(dataset='moons',points=800,noise=0.15,shuffle=True,factor=0.8):
        def change_callback():
            with plt_output:
                plt.scatter(x_axis, y_axis, c=color.reshape(-1))
                clear_output(wait=True)
                plt.show();

        if dataset not in ('moons','circles','S'):
            dataset = 'moons'

        if dataset == 'S':
            X,y = data.updateData(datasets.make_s_curve(points,noise=noise)).getAllData()
            x_axis,y_axis,color = X[:,0],X[:,2],y
        elif dataset=='moons':
            X,y = data.updateData(datasets.make_moons(points,noise=noise,shuffle=shuffle)).getAllData()
            x_axis,y_axis,color = X[:,0],X[:,1],y
        else:
            X,y = data.updateData(datasets.make_circles(points,noise=noise,shuffle=shuffle,factor=factor)).getAllData()
            x_axis,y_axis,color = X[:,0],X[:,1],y

        change_callback()

    X,y = data.getAllData()
    x_axis, y_axis, color = X[:,0], X[:,1], y 
    plt_output = widgets.Output()

    options = interactive(update_dataset_plot, 
                        dataset=Dropdown(options=['moons','S','circles'],value='moons'),
                        points=IntSlider(min=100,max=1000,step=50,value=800,continuous_update=False),
                        noise=FloatSlider(min=0.0,max=1,step=0.01,value=0.15,continuous_update=False),
                        factor=FloatSlider(min=0,max=0.99,step=0.01,value=0.8,continuous_update=False),
                        shuffle=Checkbox(value=True))

    box_layout = Layout(display='flex', flex_flow='row', align_items='center')
    dataset_widget = HBox(children=(plt_output,options), layout=box_layout)
    return dataset_widget

In [3]:
def create_network_build_widget():
    def add_layer(event):
        if not layer_boxes or (layer_boxes and layer_boxes[-1].value != 0):
            text = widgets.IntText(min=1,value=1)
            text.layout.width='50px'
            text.observe(on_layer_change,names='value')
            layer_boxes.append(text)
            children = (input_box,*layer_boxes,output_box,add_button)
            layers_widget.children = children

    def build_network(event):
        global epoch_flag
        X_train,y_train = data.getTrainData()
        net.reloadNetwork(data)
        net.setHiddenLayerSize([box.value for box in layer_boxes if box.value > 0])
        net.buildNetwork()
        if net.readyToTrain:
            net.injectTrainData(X_train, y_train)
            epoch_flag = True
            
    def on_layer_change(change):
        for box in layer_boxes:
            if box.value < 0 : box.value = 0
        with graph_output:
            clear_output(wait=True)
            hidden_layer_size = [box.value for box in layer_boxes if box.value > 0]
            net.setHiddenLayerSize(hidden_layer_size)
            net.reloadNetwork(data)
            net.showNetworkArch()


    def plot_graph():
        global epoch
        global accuracy
        global display_widgets
        while epoch_flag:
            net.forwardPass()
            _cost = calculateCrossentropyCost(net.y, net.op, net.noOfTrainingSamples, net.weight_dict)
            _accuracy = calculateAccuracy(net.y,net.op)
            net.update_weights()
            accuracy.append(_accuracy)
            epoch.append(len(accuracy))
            epoch_widget = widgets.HTML(f"<h3>EPOCH: {epoch[-1]}</h3>&nbsp;")
            cost_widget = widgets.HTML(f"<h3>COST: {_cost}</h3>&nbsp;")
            accuracy_widget = widgets.HTML(f"<h3>ACCURACY: {_accuracy}</h3>")
            display_widgets.children = (epoch_widget, cost_widget, accuracy)
            time.sleep(0.05)


    def change_flag(event):
        global epoch_flag
        epoch_flag = False
    
    accuracy = []
    epoch = []

    layer_boxes = []
    graph_output = widgets.Output()
    
    add_button = widgets.Button(description='+', button_style='info', tooltip='Add one layer')
    add_button.layout.width = '30px'
    add_button.on_click(add_layer)
    
    input_box = widgets.IntText(value=2,disabled=True)
    output_box = widgets.IntText(value=1,disabled=True)
    input_box.layout.width = '50px'
    output_box.layout.width = '50px'
    
    build_button = Button(description='Inject Data and start training', disabled=False,
                           button_style='info', 
                           tooltip='Click to build network')
    build_button.layout.width = '250px'
    build_button.on_click(build_network)
    
    layers_widget = HBox(children = [input_box,output_box,add_button])
    layers_widget.observe(on_layer_change)
    
    build_network_widget = VBox(children=(layers_widget,graph_output,build_button))
    
    stop_button = Button(icon='stop')
    stop_button.on_click(change_flag)
    epoch_widget = widgets.HTML("<h3>EPOCH: --</h3>&nbsp;")
    cost_widget = widgets.HTML("<h3>COST: --</h3>&nbsp;")
    accuracy_widget = widgets.HTML("<h3>ACCURACY: --</h3>")
    display_widgets = HBox(children=(epoch_widget, cost_widget, accuracy_widget))
    return build_network_widget

In [54]:


# def plot_widget():
#     x_scale = bqplot.LinearScale()
#     y_scale = bqplot.LinearScale()

#     x_axis = bqplot.Axis(
#         label = 'Epoch',
#         scale = x_scale
#         )

#     y_axis = bqplot.Axis(
#         label = 'Accuracy',
#         scale = y_scale,
#         orientation = 'vertical'
#         )

#     chart_accuracy = bqplot.Lines(
#         x=epoch,
#         y=accuracy,
#         scales={'x': x_scale, 'y': y_scale}

#     )

#     fig_accuracy = bqplot.Figure(
#         layout=Layout(width='400px',height='300px'),
#         axes=[x_axis,y_axis],
#         marks=[chart_accuracy],
#         fig_margin=dict(top=10,bottom=40,left=50,right=10)
#     )


    # def plot_graph():
    #     global epoch
    #     global accuracy
    #     while flag:
    #         accuracy.append(np.random.randint(1000))
    #         epoch.append(len(accuracy))
    #         line.y = accuracy
    #         line.x = epoch

    #         time.sleep(0.05)

#     stop_button = Button(icon='stop')
#     stop_button.on_click(change_flag)
#     epoch_widget = widgets.HTML("<h3>EPOCH: --</h3>&nbsp;")
#     cost_widget = widgets.HTML("<h3>COST: --</h3>&nbsp;")
#     accuracy_widget = widgets.HTML("<h3>ACCURACY: --</h3>")
#     display_widgets = HBox(children=(epoch_widget, cost_widget, accuracy_widget))
#     app = VBox(children=(display_widgets, fig_accuracy, stop_button))
#     return app

#     display(app)


In [2]:
def render_widgets():
    widget_tree = []
    widget_tree.append(create_dataset_widget())
    widget_tree.append(create_network_build_widget())
    widget_tree.append(plot_widget())
    main = VBox(children=widget_tree)
    clear_output(wait=True)
    thread = threading.Thread(target=plot_graph)
    display(main)
    thread.start()
#     thread.join()

In [3]:
render_widgets()

NameError: name 'create_dataset_widget' is not defined